In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import numpy as np
import pandas as pd
import os
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"


In [ ]:
# Build connection
spark = SparkSession.builder. \
    master("local[*]"). \
    appName("PySpark"). \
    config("spark.driver.memory","16g"). \
    config("spark.driver.maxResultSize", "4g"). \
    getOrCreate()

In [ ]:
# Load Dataset
data_path = 'data/convincing_data.csv'
data = spark.read.option("header",True).csv(data_path).rdd

In [ ]:
data.take(1)

In [ ]:
def score_label(score):
    if score>0: return 1
    elif score<0: return -1
    else: return 0
        
id_label_content = data.map(lambda x: (x['id'], score_label(float(x['sentiment'])), x['body_cleaned']))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluation(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f1 = f1_score(y, y_pred, average='weighted')
    return accuracy, precision, recall, f1

### AFINN lexicon

In [ ]:
# !pip install afinn
from afinn import Afinn
afinn = Afinn()
id_label_score_afinn = id_label_content.map(lambda x: (x[0], x[1], afinn.score(x[2])))   
id_true_pred_afinn = id_label_score_afinn.map(lambda x: (x[0], x[1], score_label(x[2])))


In [ ]:
%%time
res_afinn = pd.DataFrame(id_true_pred_afinn.collect())
print('Accuracy, Precision, Recall, F1:', evaluation(res_afinn.iloc[:,1],res_afinn.iloc[:,2]))

### VADER lexicon

In [ ]:
# !pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
id_label_score_vader = id_label_content.map(lambda x: (x[0], x[1], analyzer.polarity_scores(x[2])['compound']))  
id_true_pred_vader = id_label_score_vader.map(lambda x: (x[0], x[1], score_label(x[2])))


In [ ]:
%%time
res_vader = pd.DataFrame(id_true_pred_vader.collect())
print('Accuracy, Precision, Recall, F1:', evaluation(res_vader.iloc[:,1],res_vader.iloc[:,2]))

### Hu and Liu Lexicon

In [ ]:
positive_words = pd.read_csv('https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt',
                        names=['word'], comment=';', encoding='latin-1')['word'].tolist()
negative_words = pd.read_csv('https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt',
                        names=['word'], comment=';', encoding='latin-1')['word'].tolist()

def get_sentiment_huliu(words):
    # Define variables to keep track of the positive and negative scores
    pos_score = 0
    neg_score = 0
    
    # Loop through each word and check if it's in the positive or negative word list
    for word in words:
        if word in positive_words:
            pos_score += 1
        elif word in negative_words:
            neg_score += 1
    
    # Calculate the sentiment score for the text
    if pos_score > neg_score:
        return 1
    elif pos_score < neg_score:
        return -1
    else:
        return 0

id_true_pred_huliu = id_label_content.map(lambda x: (x[0], x[1], get_sentiment_huliu(x[2])))


In [ ]:
%%time
res_huliu = pd.DataFrame(id_true_pred_huliu.collect())
print('Accuracy, Precision, Recall, F1:', evaluation(res_huliu.iloc[:,1],res_vader.iloc[:,2]))

In [ ]:
spark.close()

### SentiWordNet Lexicon

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize

def get_sentiment_nltk(words):
    pos_score = 0
    neg_score = 0
    obj_score = 0
    for word in words:
        synsets = wn.synsets(word)
        if synsets:
            swn_synset = swn.senti_synset(synsets[0].name())
            pos_score += swn_synset.pos_score()
            neg_score += swn_synset.neg_score()

    # normalize the scores
    if pos_score > neg_score:
        return 1
    elif pos_score < neg_score:
        return -1
    else:
        return 0

id_true_pred_nltk = id_label_content.map(lambda x: (x[0], x[1], get_sentiment_nltk(x[2])))


In [ ]:
data = pd.read_csv(data_path)
data = data[['id','sentiment','body_cleaned']]